In [1]:
import tensorflow as tf
import numpy as np
import matplotlib
import os
from sklearn.preprocessing import MinMaxScaler
from matplotlib import font_manager, rc

font_name = font_manager.FontProperties(fname="/usr/share/fonts/truetype/nanum/NanumGothic_Coding_Bold.ttf").get_name()
rc('font', family=font_name)

%matplotlib tk

tf.set_random_seed(777)  # reproducibility

if "DISPLAY" not in os.environ:
    # remove Travis CI Error
    matplotlib.use('Agg')

import matplotlib.pyplot as plt

In [2]:
# train Parameters
seq_length = 9
data_dim = 4
hidden_dim = 10
output_dim = 1
learning_rate = 0.01
iterations = 901

scaler = MinMaxScaler(feature_range=(0,1))
scaler2 = MinMaxScaler(feature_range=(0, 1))

# Open, High, Low, Volume, Close
xy = np.loadtxt('./v2data/cct/train_v2_data.csv', delimiter=',')
x = scaler.fit_transform(xy[:, 0:-1])
y = scaler2.fit_transform(xy[:, [-1]])  # Close as label

# build a dataset
dataX = []
dataY = []

for i in range(0, len(x) - seq_length):
    _x = x[i:i + seq_length]
    _y = y[i + seq_length]  # Next close price
    dataX.append(_x)
    dataY.append(_y)

xy2 = np.loadtxt('./sun_data/2018-02-27.csv',delimiter=',')

x2 = scaler.fit_transform(xy2) # 데이터 없을때
# x2 = scaler.fit_transform(xy2[:, 0:-1]) # 데이터 있을때
# y2 = scaler2.fit_transform(xy2[:, [-1]]) #데이터 있을때

y2 = np.loadtxt('./v2data/cct/2018-02-27.csv',delimiter=',')

dataX2 = []
dataY2 = [] #데이터 있을때

for i in range(0, len(x2) - seq_length):
    _x2 = x2[i:i+seq_length]
    _y2 = y2[i] #데이터 있을때
    dataX2.append(_x2)
    dataY2.append(_y2) #데이터 있을때
# train/test split

train_size = len(dataX)
test_size = len(dataX2)

trainX, testX = np.array(dataX[0:train_size]),np.array(dataX2[0:test_size])
trainY = np.array(dataY[0:train_size])

In [3]:
# input place holders
X = tf.placeholder(tf.float32, [None, seq_length, data_dim])
Y = tf.placeholder(tf.float32, [None, 1])

# build a LSTM network
cell = tf.contrib.rnn.BasicLSTMCell(
    num_units=hidden_dim, state_is_tuple=True, activation=tf.tanh)
outputs, _states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
Y_pred = tf.contrib.layers.fully_connected(
    outputs[:, -1], output_dim, activation_fn=None)  # We use the last cell's output

# cost/loss
loss = tf.reduce_sum(tf.square(Y_pred - Y))  # sum of the squares
# optimizer
optimizer = tf.train.AdamOptimizer(learning_rate)
train = optimizer.minimize(loss)

Instructions for updating:
Use the retry module or similar alternatives.


In [4]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)

    # Training step
    for i in range(iterations):
        _, step_loss = sess.run([train, loss], feed_dict={
                                X: trainX, Y: trainY})
        print("[step: {}] loss: {}".format(i, step_loss))
    # Test step
    test_predict = sess.run(Y_pred, feed_dict={X: testX})
    
    test_predict = scaler2.inverse_transform(test_predict)

[step: 0] loss: 625.465576171875
[step: 1] loss: 243.216064453125
[step: 2] loss: 127.95773315429688
[step: 3] loss: 151.13388061523438
[step: 4] loss: 161.937744140625
[step: 5] loss: 126.19656372070312
[step: 6] loss: 76.03811645507812
[step: 7] loss: 39.82243347167969
[step: 8] loss: 27.3908748626709
[step: 9] loss: 34.174259185791016
[step: 10] loss: 48.08057403564453
[step: 11] loss: 57.171363830566406
[step: 12] loss: 56.395240783691406
[step: 13] loss: 47.802181243896484
[step: 14] loss: 36.51506423950195
[step: 15] loss: 27.30362892150879
[step: 16] loss: 22.87127685546875
[step: 17] loss: 23.444726943969727
[step: 18] loss: 27.228235244750977
[step: 19] loss: 31.47844123840332
[step: 20] loss: 33.84758758544922
[step: 21] loss: 33.35789108276367
[step: 22] loss: 30.517086029052734
[step: 23] loss: 26.7100772857666
[step: 24] loss: 23.387014389038086
[step: 25] loss: 21.474365234375
[step: 26] loss: 21.162368774414062
[step: 27] loss: 22.0185604095459
[step: 28] loss: 23.289947

[step: 228] loss: 17.856651306152344
[step: 229] loss: 17.85490608215332
[step: 230] loss: 17.8531494140625
[step: 231] loss: 17.851383209228516
[step: 232] loss: 17.849611282348633
[step: 233] loss: 17.84782600402832
[step: 234] loss: 17.846033096313477
[step: 235] loss: 17.844228744506836
[step: 236] loss: 17.8424129486084
[step: 237] loss: 17.840593338012695
[step: 238] loss: 17.838764190673828
[step: 239] loss: 17.836925506591797
[step: 240] loss: 17.835073471069336
[step: 241] loss: 17.833209991455078
[step: 242] loss: 17.83133888244629
[step: 243] loss: 17.829458236694336
[step: 244] loss: 17.827571868896484
[step: 245] loss: 17.825666427612305
[step: 246] loss: 17.823759078979492
[step: 247] loss: 17.82183837890625
[step: 248] loss: 17.819910049438477
[step: 249] loss: 17.817970275878906
[step: 250] loss: 17.816022872924805
[step: 251] loss: 17.81406021118164
[step: 252] loss: 17.812091827392578
[step: 253] loss: 17.81011199951172
[step: 254] loss: 17.808120727539062
[step: 255]

[step: 452] loss: 17.421443939208984
[step: 453] loss: 17.418956756591797
[step: 454] loss: 17.416423797607422
[step: 455] loss: 17.413843154907227
[step: 456] loss: 17.41122055053711
[step: 457] loss: 17.40854835510254
[step: 458] loss: 17.405826568603516
[step: 459] loss: 17.403047561645508
[step: 460] loss: 17.400217056274414
[step: 461] loss: 17.3973331451416
[step: 462] loss: 17.39438819885254
[step: 463] loss: 17.391387939453125
[step: 464] loss: 17.38832664489746
[step: 465] loss: 17.385196685791016
[step: 466] loss: 17.382009506225586
[step: 467] loss: 17.37874984741211
[step: 468] loss: 17.375423431396484
[step: 469] loss: 17.372032165527344
[step: 470] loss: 17.368566513061523
[step: 471] loss: 17.365026473999023
[step: 472] loss: 17.361412048339844
[step: 473] loss: 17.357717514038086
[step: 474] loss: 17.353944778442383
[step: 475] loss: 17.350095748901367
[step: 476] loss: 17.346158981323242
[step: 477] loss: 17.342138290405273
[step: 478] loss: 17.338035583496094
[step: 4

[step: 677] loss: 12.08152961730957
[step: 678] loss: 12.102556228637695
[step: 679] loss: 12.22868537902832
[step: 680] loss: 12.384658813476562
[step: 681] loss: 12.494133949279785
[step: 682] loss: 12.322273254394531
[step: 683] loss: 12.039268493652344
[step: 684] loss: 11.888249397277832
[step: 685] loss: 11.95478343963623
[step: 686] loss: 12.156081199645996
[step: 687] loss: 12.309076309204102
[step: 688] loss: 12.30502986907959
[step: 689] loss: 11.984086990356445
[step: 690] loss: 11.730917930603027
[step: 691] loss: 11.75709342956543
[step: 692] loss: 11.992030143737793
[step: 693] loss: 12.252853393554688
[step: 694] loss: 12.122190475463867
[step: 695] loss: 11.793021202087402
[step: 696] loss: 11.552664756774902
[step: 697] loss: 11.65514850616455
[step: 698] loss: 11.976968765258789
[step: 699] loss: 12.015149116516113
[step: 700] loss: 11.822999954223633
[step: 701] loss: 11.47487735748291
[step: 702] loss: 11.384262084960938
[step: 703] loss: 11.56360149383545
[step: 70

In [5]:
# Plot predictions
plt.title('2018-02-27 predict')
plt.plot(dataY2, label='row data') # 데이터 있을때
plt.plot(test_predict, label='predict')
plt.xlabel("Time Period")
plt.ylabel("CCT")
# plt.ylim(0, 140000)
plt.xticks(np.arange(0, 901, step = 56), ['5', '6', '7', '8', '9', '10', '11', '12', '1', '2', '3', '4', '5', '6', '7', '8']) # v2 모델
plt.yticks(np.arange(0, 50000, step=5000))
plt.grid(True)
plt.legend(loc='upper right')
plt.show()